In [20]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

In [21]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)
    
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from PIL import Image
from matplotlib.pyplot import plot

IMG_PATH = "maps/basic.png"
LARGURA_IMAGEM = 80
WORLD_ORIGIN = (-4.0, -4.0)
LARGURA_PIXEL_MUNDO = 0.1 # Valor da largura de 1px da imagem geradora do mapa em coordenadas do mundo
#Para isso, ao gerar o mapa: uma imagem 80x80px deve ser importada com x-size=8m e y-size=8m

def load_binary_mask():
    img = Image.open(IMG_PATH).convert("L")
    arr = np.array(img, dtype=np.uint8)
    arr = 255 - arr
    mask = arr < 200
    return mask

def px_to_world(col_px, row_px):
    row_from_bottom = (LARGURA_IMAGEM-1) - row_px
    offset = LARGURA_PIXEL_MUNDO
    offset = 0
    x = WORLD_ORIGIN[0] + (col_px + offset) * (LARGURA_PIXEL_MUNDO)
    y = WORLD_ORIGIN[1] + (row_from_bottom + offset) * (LARGURA_PIXEL_MUNDO)
    return float(x), float(y)

def world_to_px(x, y):
    col_px = int((x - WORLD_ORIGIN[0]) / LARGURA_PIXEL_MUNDO)
    row_px = int((y - WORLD_ORIGIN[1]) / LARGURA_PIXEL_MUNDO)
    return int(col_px), int(row_px)

def plot_binary_mask(mask):
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(8, 8))
    plt.imshow(mask, cmap='binary')
    plt.title('Mapa Binário')
    plt.xlabel('Coluna (pixels)')
    plt.ylabel('Linha (pixels)')
    plt.show()

mask = load_binary_mask()

# Adiciona um sistema de coordenadas em cada célula do mapa que tem um obstáculo
# para visualizar no simulador se os pontos estão corretos
teste_dos_pontos_dos_obstaculos = '''
def print_mask_coordinates(mask):
    for row in range(mask.shape[0]):
        for col in range(mask.shape[1]):
            if mask[row,col]:  # Se for True (obstáculo)
                x, y = px_to_world(col, row)
                # Adiciona um ReferenceFrame na coordenada do obstáculo
                frame_name = f"ReferenceFrame_{col}_{row}"
                frame_handle = sim.createDummy(0.1)  # Tamanho do frame = 0.1m
                sim.setObjectAlias(frame_handle, frame_name)
                sim.setObjectPosition(frame_handle, -1, [x, y, 0.01])  # z=0.01 para ficar levemente acima do chão
                print(f"Obstáculo em: Pixel({col},{row}) -> Mundo({x:.2f},{y:.2f})")

print("\nCoordenadas dos obstáculos:")
'''
#plot_binary_mask(mask)

In [23]:
try:
    # Handle para o ROBÔ
    robotname = 'robotino'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    wheel1 = sim.getObject('/' + robotname + '/wheel0_joint')
    wheel2 = sim.getObject('/' + robotname + '/wheel1_joint')
    wheel3 = sim.getObject('/' + robotname + '/wheel2_joint')
    
    # Dados Robotino
    L = 0.135   # Metros
    r = 0.040   # Metros
               
    # Cinemática Direta
    Mdir = np.array([[-r/np.sqrt(3), 0, r/np.sqrt(3)], [r/3, (-2*r)/3, r/3], [r/(3*L), r/(3*L), r/(3*L)]])

    # Configuração inicial (x, y, w)
    q = np.array([0, 0, 0])

    # Goal configuration (x, y, theta)    
    qgoal = np.array([3, 3, np.deg2rad(90)])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()


    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0], sim.handle_world)
    sim.setObjectOrientation(goalFrame, [0, 0, qgoal[2]], sim.handle_world)
    gain = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]])
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        ori = sim.getObjectOrientation(robotHandle, sim.handle_world)

        q = np.array([pos[0], pos[1], ori[2]])
        
        error = qgoal - q
        print(f"Error: {np.linalg.norm(error[:2])}")
        
        # Margem aceitável de distância
        if (np.linalg.norm(error[:2]) < 0.05):
            break

        # Controller
        qdot = gain @ error
        
        # Cinemática Inversa
        # w1, w2, w3
        Minv = np.linalg.inv(Rz(q[2]) @ Mdir)
        u = Minv @ qdot
        
        # Enviando velocidades
        sim.setJointTargetVelocity(wheel1, u[0])
        sim.setJointTargetVelocity(wheel2, u[1])
        sim.setJointTargetVelocity(wheel3, u[2]) 
                
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(wheel1, 0)
    sim.setJointTargetVelocity(wheel2, 0)
    sim.setJointTargetVelocity(wheel3, 0)
    
    print('CALC Pos: ', sim_time, q[:2], np.rad2deg(q[2]))
      
    pos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('SIM Pos: ', pos)

    ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
    print('SIM Ori: ', np.rad2deg(ori))
    

except Exception as e:
    print(f"An error occurred: {e}")

Starting robot control loop...
Simulation time: 0.05 [s]
Error: 8.36010212038732
Simulation time: 0.10 [s]
Error: 8.355108047777673
Simulation time: 0.15 [s]
Error: 8.347340972638182
Simulation time: 0.20 [s]
Error: 8.337136528504722
Simulation time: 0.25 [s]
Error: 8.324727099417446
Simulation time: 0.30 [s]
Error: 8.310047353873976
Simulation time: 0.35 [s]
Error: 8.293777757174219
Simulation time: 0.40 [s]
Error: 8.276076812345034
Simulation time: 0.45 [s]
Error: 8.256961417143591
Simulation time: 0.50 [s]
Error: 8.2369112552678
Simulation time: 0.55 [s]
Error: 8.216652464229172
Simulation time: 0.60 [s]
Error: 8.19546781495812
Simulation time: 0.65 [s]
Error: 8.173157016956331
Simulation time: 0.70 [s]
Error: 8.150415478060127
Simulation time: 0.75 [s]
Error: 8.1270215023536
Simulation time: 0.80 [s]
Error: 8.103287958915805
Simulation time: 0.85 [s]
Error: 8.079460083844411
Simulation time: 0.90 [s]
Error: 8.055226673629225
Simulation time: 0.95 [s]
Error: 8.030288481546663
Simula

KeyboardInterrupt: 

In [ ]:
# Parando a simulação
sim.stopSimulation()
print('Program ended')

Program ended
